# Using a SurrealDB Knowedge Graph and DeepSeek to build a Graph-RAG 

This Jupyter Notebook illustrates the implementation and utilization of a graph-based Retrieval-Augmented Generation (RAG) system using a knowledge graph stored in SurrealDB. It focuses on leveraging the knowledge graph created by the graph_extractor.ipynb notebook for question answering tasks.

### Key Functionality
####  Data Loading:
        Loads the raw text data from which will be used for context in the prompt.

#### Leveraging the Existing Knowledge Graph:

        Create a SurrQL function to query the knowledge graph using vector search for the descriptions of the entities and the other entities and relationships they have in the graph. This data will also be fed into the prompt for context.

        
#### Response Generation without a knowledge graph:
    Using a template prompt in the RAG_prompts.py file a simple context window of the text data loaded earlier with the question is asked of the LLM model.

    

#### Response Generation with knowledge graph:
    Using a template prompt in the RAG_prompts.py file a context window of the text data loaded earlier, plus the graph data retrieved from SurrealDB with the question is asked of the LLM model.


##### notes:
    This notebook utilizes libraries :
        surrealdb to interact with SurrealDB
        ollama to interact with DeepSeek

    Prerquisites:
            load an embedding model into Surreal: https://github.com/apireno/surrealDB_embedding_model
            load a knowledge graph into Surreal: (graph_extractor.ipynb in this Repo)
            install ollama: https://ollama.com/download
            install the ollama python client: pip install ollama
            

    The notebook is loosely based on a larger repository microsoft/graphrag. The microsoft/graphrag repository provides a more comprehensive, modular graph-based RAG system. This notebook focuses specifically on the knowledge graph construction aspect.

In [2]:
import sys
import os
import json
from ollama import chat,generate,ChatResponse,GenerateResponse
import os
import time
import ipynb_path
import pandas as pd
import re
from surrealdb import AsyncSurrealDB

from IPython.display import display, Markdown,JSON
from IPython.display import JSON


#get this notebook's path for access to the other files needed
dir_path = os.path.dirname(os.path.realpath(ipynb_path.get(__name__)))
sys.path.append(dir_path) #add the current directory for adding py imports

#This python file holds the prompt templates
from RAG_prompts import DEEPSEEK_STD_RAG_PROMPT, DEEPSEEK_GRAPH_RAG_PROMPT

In [3]:

# this folder
nb_folder = dir_path
input_file = nb_folder + "/Operation Dulce v2 1 1.md"

out_folder = nb_folder + "/deepseek_rag_{0}".format(time.strftime("%Y%m%d-%H%M%S"))
os.makedirs(out_folder, exist_ok=True)
chat_file = out_folder + "/chat.json"


#this variable is used to log debugging info
messages_to_log = []


#the SurrealDB hosting information
ip = "0.0.0.0:8000"
url = "ws://{0}".format(ip)

u = "root"
p = "root"
n = "graph_rag"
d = "graph_rag"
db_folder = nb_folder + "/db"

surrealdb_start = "surreal start --allow-net --log none --user {u} --pass {p} --bind {ip} \"rocksdb://{db_folder}\"".format(
    u=u,
    p=p,
    ip=ip,
    db_folder=db_folder)

print("""
run this command if your surreal instance isn't running yet 
copy and paste from below into a terminal
""")

print(surrealdb_start)

#and ensure you installed the embedding model!
#the model will power the function fn::sentence_to_vector($text)
print("""
ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)

Also ensure ollama is running and your have pulled your ollama model:

ollama pull <model_name>

For instance this will run the smallest 1.5b model:

ollama pull deepseek-r1:1.5b

A full list of the Deepseek models available can be found here:
    https://ollama.com/library/deepseek-r1
    
"""   )  





run this command if your surreal instance isn't running yet 
copy and paste from below into a terminal

surreal start --allow-net --log none --user root --pass root --bind 0.0.0.0:8000 "rocksdb:///Users/sandro/git_repos/graph_rag/db"

ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)

Also ensure ollama is running and your have pulled your ollama model:

ollama pull <model_name>

For instance this will run the smallest 1.5b model:

ollama pull deepseek-r1:1.5b

A full list of the Deepseek models available can be found here:
    https://ollama.com/library/deepseek-r1
    



In [4]:

# get the text from the source file for use in our prompt
with open(input_file, "r") as f:
    input_text = f.read()


entity_types = ["PERSON", "PLACE"]
relation_type = "RELATED_TO"

#create the function that will return the portion of the knowledge graph 
#relevant to the question using vector search on the description embedding
#we will use python format to replace the table names in our funtion with the entity types and relation type strings
retrieve_function_surql = """
DEFINE FUNCTION OVERWRITE fn::get_related_entities($text: string) {{
    
    LET $emb = fn::sentence_to_vector($text);
    
    RETURN SELECT id,description, ->{1}.{{out,strength,description}} FROM {0} 
        WHERE  embedding <|5,COSINE|> $emb;
}};
""".format(entity_types[0],relation_type)

#execute the DDL of the function
async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    outcome = await db.query(retrieve_function_surql)



In [5]:

#This is a helper function to make the knowledge graph a bit easier to read.
def transform_record_ids(data):
  """
  Transforms the input data to extract the 'ID' field from 'out' and 'in' objects.

  Args:
    data: An array of dictionaries.

  Returns:
    The transformed array of dictionaries.
  """
    
  for item in data:
    if "id" in item:
        item["id"] = item["id"].id
    if "->RELATED_TO" in item:
      for inner_dict in item["->RELATED_TO"]:
        if "out" in inner_dict and hasattr(inner_dict["out"], "id"):
          inner_dict["out"] = inner_dict["out"].id
        if "in" in inner_dict and hasattr(inner_dict["in"], "id"):
          inner_dict["in"] = inner_dict["in"].id
        if "id" in inner_dict and hasattr(inner_dict["id"], "id"):
          inner_dict["id"] = inner_dict["id"].id
      
    
  return data


In [6]:

#Let's test out vector/graph query
#we should see a collection of people and relationships
#where the person's description is related to the question 

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    outcome = await db.query(get_the_graph_context_surql.format("Who is the main character?"))

result = transform_record_ids(outcome[0]["result"])


print(json.dumps(result, indent=4, sort_keys=True))
    

[
    {
        "->RELATED_TO": [
            {
                "description": "Alex Mercer operates within Dulce Base during Operation: Dulce.",
                "out": "DULCE BASE",
                "strength": 9
            },
            {
                "description": "Alex Mercer and Taylor Cruz are both members of the Paranormal Military Squad and work together on Operation: Dulce.",
                "out": "TAYLOR CRUZ",
                "strength": 8
            },
            {
                "description": "Alex Mercer reports to and receives orders from Washington D.C.",
                "out": "WASHINGTON",
                "strength": 6
            },
            {
                "description": "Alex Mercer and Jordan Hayes are both members of the Paranormal Military Squad and work closely together, sharing a mutual respect and understanding.",
                "out": "JORDAN HAYES",
                "strength": 9
            },
            {
                "description": "Al

In [7]:
#Lets test out our ollama hosted deepseek model
response: GenerateResponse = generate(model='deepseek-r1:1.5b', prompt= "What are the top 3 things to do in New York?")

#there's a lot of content returned here. the response object is the one we want...
print(response)

#notice that <think> section?
print(response.response)

model='deepseek-r1:1.5b' created_at='2025-01-29T15:35:44.780474Z' done=True done_reason='stop' total_duration=12311924000 load_duration=853846750 prompt_eval_count=16 prompt_eval_duration=834000000 eval_count=870 eval_duration=10622000000 response="<think>\nOkay, so I need to figure out the top three things to do in New York. Hmm, I'm not super familiar with all the options, but let me think through this step by step.\n\nFirst, maybe I should consider what people typically do when they visit a place like New York. They might be hiking trails, visiting museums, eating great food, and experiencing different cultures. That makes sense because NYC has so much to offer in terms of activities.\n\nThe user mentioned the top three things, but it's not clear which ones are most commonly associated with NYC. I know that NYC is known for its diverse geography, so maybe being a tourist favorite includes seeing the Statue of Liberty and Central Park. Those are definitely top priorities because they

In [71]:

#Let's create a helper function to pull out the <think> so we can do formatting and what not on the rest of the resposne
def parse_deepseek_response(input_string):
  """
  Parses a string containing <think> tags and extracts the content.

  Args:
    input_string: The string to parse.

  Returns:
    A dictionary with two keys:
      - "think": The content between the <think> tags.
      - "content": The rest of the string.
  """

  # Use regular expression to find the content within the <think> tags
  think_match = re.search(r"<think>(.*?)</think>", input_string, re.DOTALL)
  think_content = think_match.group(1).strip() if think_match else ""

  # Remove the <think> section from the original string
  content = re.sub(r"<think>.*?</think>\s*", "", input_string, flags=re.DOTALL).strip()

  return {"think": think_content, "content": content}

# Example usage
input_string = """
<think>

</think>

The color of the sky we see during the day is a result of the atmospheric absorption and scattering of sunlight. The short wavelength blue light, which is more easily absorbed by the atmosphere, passes through larger particles in the atmosphere, such as air molecules, before reaching us. At night, when the sun has set and the atmosphere is thinner, other wavelengths are scattered toward us by tiny particles like water droplets and small dust particles in the air. This scattering of light gives us a different color for the sky at night.
"""

result = parse_deepseek_response(input_string)
print(result)

{'think': '', 'content': 'The color of the sky we see during the day is a result of the atmospheric absorption and scattering of sunlight. The short wavelength blue light, which is more easily absorbed by the atmosphere, passes through larger particles in the atmosphere, such as air molecules, before reaching us. At night, when the sun has set and the atmosphere is thinner, other wavelengths are scattered toward us by tiny particles like water droplets and small dust particles in the air. This scattering of light gives us a different color for the sky at night.'}


In [73]:
#these functions will call ollama api based on our prompt templates and input


#input string of the model name and the prompt content
def generate_content_with_attachment(the_model,the_prompt):
    response: GenerateResponse = generate(model=the_model, prompt=the_prompt)
    parsed_response = parse_deepseek_response(response.response)
    return {"response":response, "think": parsed_response["think"],"content":parsed_response["content"]}
             



#this generates the prompt for a std RAG approach where we are supplying the document with the question
def generate_std_LLM_prompt(the_question,the_document_text):
    messages = DEEPSEEK_STD_RAG_PROMPT.format(
                question = the_question,
                documents = the_document_text
            )
    return messages


#this function will call the LLM with a std RAG approach prompt 
def get_std_LLM_response(the_model,the_question,the_document_text):
    
    messages = generate_std_LLM_prompt(the_question,the_document_text)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
        
    response = generate_content_with_attachment(the_model,messages)
    return response

   
#this generates the prompt for a graphRAG approach where we are supplying the document and knowledge graph with the question
#it calls the SurQL function get_related_entities to do a vector search based on the input question
async def generate_graph_RAG_prompt(the_db,the_question,the_document_text):
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    
    outcome = await the_db.query(get_the_graph_context_surql.format(the_question))
    messages =  DEEPSEEK_GRAPH_RAG_PROMPT.format(
                question = the_question,
                knowledge_graph = outcome,
                documents = the_document_text
            )
    
    return messages
    
    
#this function will call the LLM with a prompt that includes the text of the original document
#plus the relevant entities and relationships from the knowledge graph based on a vector search
async def get_graph_RAG_response(the_db,the_model,the_question,the_document_text):
    
    messages = await generate_graph_RAG_prompt(the_db,the_question,the_document_text)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
    
    response = generate_content_with_attachment(the_model,messages)
    return response
    
       

In [77]:

#specify the model to use

#model = "deepseek-r1:1.5b"
model = "deepseek-r1:7b"


#retreive the input document to a string
with open(input_file) as f:
    document_text = f.read()




In [78]:
#Let's see how the prompts will look
#You can copy and paste these into any LLM chat engine to test

q = "Who is the main character and what motivates them?"

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
messages = generate_std_LLM_messages(q,"document_text")


print(messages)

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    messages = await generate_graph_RAG_messages(db,q,"document_text")
    print(messages)
 
   






 ------ STD RAG -------------------------





-Goal-
You are a researcher who is providing answers to a question for your client based on documents provided you. Answer the questions to the best of your ability given the documents. The document text will be provided between the <document> delimiters, the knowledge graph between the  <knowledge graph> delimiters, and the prompt question between the <question> delimiters.

This is the data to respond to:
<documents>
document_text
</documents>
<question>
Who is the main character and what motivates them?
</question>

 




 ------ GRAPH RAG -------------------------





-Goal-
You are a researcher who is providing answers to a question for your client based on documents provided you. You also have access to a knowledge graph based on the documents provided. Answer the questions to the best of your ability given the documents and knowledge graph. The document text will be provided between the <document> delimiters, the knowledge grap

In [96]:
#Lets test a question...

q = "Who is the main character and what motivates them? Format your response in bullet points."


#we get the response
response = get_std_LLM_response(model,q,document_text)


print ("\n ------ Was there an error or did it complete? -------------------------\n")
#stop: This is the most common reason. It means the model generated text until it reached a natural stopping point, such as the end of a sentence or a logical conclusion. It could also indicate that the model reached a predefined maximum number of tokens.
#length: The model reached the maximum number of tokens specified in the request parameters.
#eos: The model encountered an end-of-sequence token, which signals the end of the generated text.
#error: An error occurred during the generation process. This could be due to various reasons, such as invalid input, model issues, or server problems.
#null: This usually means the generation is still in progress and hasn't finished yet.

print(response["response"].done_reason)

print ("\n ------ Think -------------------------\n")
#this is Deepseek's custom "thinking" 
print(response["think"])
print ("\n ------ Content -------------------------\n")
#For the rest of the content let's clean it up with a bit of Markdown
display(Markdown(response["content"]))



stop

 ------ Think -------------------------

Alright, I need to figure out who the main characters are in this document and what motivates each of them. Let's start by reading through the document carefully.

First, there are several people mentioned: Dr. Jordan Hayes, Agent Sam Rivera, Agent Alex Mercer, and Agent Taylor Cruz. It seems like they're all part of the Paranormal Military Squad based at Dulce Base. 

Dr. Jordan Hayes is involved in analyzing codes and patterns from extraterrestrial sources. She's described as having deep expertise, so she must be a key character driving the scientific aspect of the mission.

Agent Sam Rivera is mentioned alongside her, contributing visual data. His role seems to enhance the technological or graphical interpretation of what Dr. Hayes finds, making him a supporting but crucial ally.

Then there's Agent Alex Mercer and Agent Taylor Cruz. They take on more active roles, with Alex leading decisions based on the data they analyze together. Tay

- **Dr. Jordan Hayes**: Motivated by her desire to uncover unknown civilizations through scientific inquiry.
- **Agent Sam Rivera**: Driven by the need to provide accurate and detailed visual representations of the findings for effective communication with aliens.
- **Agent Alex Mercer**: Passionate about initiating contact and exploring uncharted territories, both literal and extraterrestrial.
- **Agent Taylor Cruz**: Motivated by caution and the need to ensure that any attempts at contact are well-informed and safe.

In [97]:
#and now let's see how the graph RAG will look...

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,document_text)
    

print ("\n ------ Was there an error or did it complete? -------------------------\n")
#stop: This is the most common reason. It means the model generated text until it reached a natural stopping point, such as the end of a sentence or a logical conclusion. It could also indicate that the model reached a predefined maximum number of tokens.
#length: The model reached the maximum number of tokens specified in the request parameters.
#eos: The model encountered an end-of-sequence token, which signals the end of the generated text.
#error: An error occurred during the generation process. This could be due to various reasons, such as invalid input, model issues, or server problems.
#null: This usually means the generation is still in progress and hasn't finished yet.

print(response["response"].done_reason)
print ("\n ------ Think -------------------------\n")
print(response["think"])
print ("\n ------ Content -------------------------\n")
display(Markdown(response["content"]))


 ------ Was there an error or did it complete? -------------------------

stop

 ------ Think -------------------------

Okay, so I need to figure out who the main character is in this story and what motivates them. The question specifically asks for a list of bullet points with the name and their motivation.

First, looking at the provided text, it's about agents working on Operation Dulce. There are mentions of Jordan Hayes, Alex Mercer, Sam Rivera, and Taylor Cruz. Each has different roles within the team. 

I know that in stories, especially action or military ones, there are often multiple characters with distinct roles. But when asked for the "main character," it's usually someone who is the protagonist—someone driving the plot forward.

Looking at each character:

- **Jordan Hayes**: He's a doctor and a key member of the team doing critical analysis. His role seems crucial as he's handling technical aspects, but does this make him the main character? It might be, but let me che

**Answer:**

- **Name**: Agent Alex Mercer  
- **Motivation**: He is the leader of the team during Operation Dulce, driving the team's actions and decisions. His determination and observativeness make him pivotal in directing the operation effectively.

In [98]:

#these helper functions will run both approaches on the same question and print them for us

async def print_response(response):
    await print_reason(response["response"].done_reason)
    await print_think(response["think"])
    await print_content(response["content"])

async def print_reason(done_reason):
    if done_reason=="stop":
        return

    else:
        print(f"""!!There was an issue generating try again!!!!!

 -----------------  {done_reason} -----------------

        
    #stop: This is the most common reason. It means the model generated text until it reached a natural stopping point, such as the end of a sentence or a logical conclusion. It could also indicate that the model reached a predefined maximum number of tokens.
    #length: The model reached the maximum number of tokens specified in the request parameters.
    #eos: The model encountered an end-of-sequence token, which signals the end of the generated text.
    #error: An error occurred during the generation process. This could be due to various reasons, such as invalid input, model issues, or server problems.
    #null: This usually means the generation is still in progress and hasn't finished yet.
        """)
    
    print(response["response"].done_reason)

async def print_think(think):
    print(f"""<think>
{think}
</think>""")


async def print_content(content):
    print ("<content>")
    display(Markdown(content))
    print ("</content>")

    
async def process_and_display_RAG_comparison(q):
    print ("""------ STD RAG -------------------------
""")
    std_response = get_std_LLM_response(model,q,document_text)
    await print_response(std_response)
    
    print ("""------ GRAPH RAG -------------------------
""")
    
    async with AsyncSurrealDB(url) as db:
        await db.sign_in( u,  p) 
        await db.use(n, d)
        graph_response = await get_graph_RAG_response(db,model,q,document_text)
        await print_response(graph_response)
       
    


In [99]:

#Let's test a question that asks about the main character's relationships
q = "What places and people does Alex Mercer hold dearest? Format your response in bullet points. Format your response in bullet points."
await process_and_display_RAG_comparison(q)



------ STD RAG -------------------------

<think>
Alright, I need to figure out what places and people Alex Mercer holds dear based on the document provided. Let me start by reading through the document carefully.

First, I'll look for mentions of Alex's personal interests or relationships. In the document, there are several references to people he admires: Taylor Cruz, Sam Rivera, Dr. Jordan Hayes, and others like the Prime Minister of Ber剔 and the CEO of Krellstar. These seem to be individuals in his field who inspire him.

Next, I'll check if Alex has any specific places that stand out. The document doesn't mention physical locations he holds dear but mentions Dulce Base, which is a fictional setting. However, given his role as an agent, it's more about the abstract concepts or people rather than real-world locations.

I also notice phrases like "the Prime Minister of Ber剔" and "the CEO of Krellstar," suggesting he admires powerful or influential figures in their respective fields. 

- Taylor Cruz  
- Sam Rivera  
- Dr. Jordan Hayes  
- The Prime Minister of Ber剔  
- The CEO of Krellstar

</content>
------ GRAPH RAG -------------------------

<think>
Okay, so the user is asking about what places and people Alex Mercer holds dear. They provided a passage with details about several military agents: Alex Mercer, Jordan Hayes, Sam Rivera, and Taylor Cruz. I need to focus on Alex and extract his relationships and preferences.

First, looking at the passage, Alex is part of the Paranormal Military Squad. He leads during Operation Dulce. The knowledge graph shows he operates within Dulce Base, which sounds like a significant location for him—maybe his favorite place because it's where he works.

Next, he mentors Sam Rivera, as mentioned in the knowledge graph. Mentoring someone you care about is usually a strong bond builder, so that should be included. Also, the passage says he has a strong sense of responsibility, which makes me think he values duty or loyalty to his team and superiors.

The question asks for places and people, so Dulce Base is definitely one place. Then, Sa

- **Place**: Dulce Base - Alex finds work and command there, making it a significant location in his life.  
- **Person**: Sam Rivera - Alex considers Sam a valuable mentor and comrade due to their shared experiences and mutual respect.  
- **Quality**: Determination and responsibility - Alex thrives on being reliable and dependable, qualities he often ascribes to himself.

</content>


In [102]:

# a bit of struture and creative calculation around his relationshps
q = """
Create a table of people that Alex communicates with:

In the first column indicate the person. 
In the 2nd column list the frequency he has scenes with them.
In the 3rd column list the places they had scenes together ordered by importance of the place and scene.
In the 4th column indicate the strength of the bond between the people.
In the 5th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""
await process_and_display_RAG_comparison(q)

------ STD RAG -------------------------

<think>
Alright, so I need to create a table based on the information given about Alex and his communications with other characters from that document. Let me try to parse this step by step. 

First, I'll start by identifying all the people mentioned who interact with Alex. From the document, I can see names like Sam Rivera, Dr. Jordan Hayes, Agent Alex Mercer (which is probably a typo since it's the same person), and Taylor Cruz. So that gives me four primary characters besides Alex himself.

Next, I need to figure out how frequently each of these people appears in scenes with Alex. The second column requires this frequency, so I'll have to count how often each pair meets. For example, if Sam appears multiple times with Dr. Hayes but only once with Taylor, that will affect their frequencies.

Then, the third column needs the places where they had scenes together, ordered by importance. This might involve looking at the document for specific lo

Here is the table summarizing Alex's interactions with other characters:

```markdown
| Person       | Frequency of Scenes Together | Locations (Ordered by Importance) | Strength of Bond | Total Weight of Interactions |
|-------------|------------------------------|-------------------------------------|------------------|-------------------------------|
| Dr. Jordan  | High                         | Dulce Base                          | Strong           | 25                            |
| Taylor      | Moderate                     | Dulce Base, Other Secondary       | Moderate         | 18                            |
| Sam         | Low                          | Specific Secondary Site            | Weak             | 9                             |
| Agent Alex   | Very High                   | Constellation Core                | Very Strong     | 36                            |
```

### Explanation:
- **Dr. Jordan**: Engaged in frequent interactions at Dulce Base, indicating a strong professional and personal bond.
- **Taylor**: Occasional but significant encounters elsewhere, reflecting moderate connections.
- **Sam**: Rare interactions, suggesting weaker bonds despite potential acquaintances.
- **Agent Alex**: Constant presence throughout, showcasing a very strong and consistent relationship.

This table aggregates the data based on interaction frequency, location importance, perceived bond strength, and overall interaction weight.

</content>
------ GRAPH RAG -------------------------

<think>
Okay, I need to create a table that lists the people Alex communicates with. The user has given me some data about several agents: Alex M mercer, Jordan Hayer, Sam RIVERA, and Taylor Cruz. All of them are part of the Paranormal Military Squad. 

First, I should figure out who Alex interacts with. From the text provided, it looks like there's a conversation between Alex, Jordan, and Sam in Dulce Base during Operation: Dulce. So Alex definitely communicates with both Jordan and Sam in this setting.

Next, for each person, I need to list how often they appear together (frequency). In the given text, all three agents are present once in that specific scene. So their frequency is 1 each.

Then, for the third column, I have to order the places where they had scenes by importance. The primary place mentioned is Dulce Base, so it should be first. There isn't another location listed elsewhere in the text, so Dulce Base is the only o

Here is a table summarizing the interactions between Alex M mercer and other members:

| Person          | Frequency of Scenes | Primary Place of Interaction | Bond Strength |
|-----------------|----------------------|------------------------------|---------------|
| Alex M mercer   | 1                    | Dulce Base                   | 10            |
| Jordan Hayer    | 1                    | Dulce Base                   | -             |
| Sam RIVERA      | 1                    | Dulce Base                   | 7             |

### Explanation:
- **Alex M mercer** communicates once with the team in **Dulce Base**, where he interacts with both **Jordan Hayer** and **Sam RIVERA**.
- Jordan Hayer has a neutral bond strength (no specific interaction strength mentioned).
- Sam RIVERA has a moderate bond strength due to Alex mentoring him.

This table focuses only on interactions between Alex M mercer and other members based on the provided text.

</content>


In [101]:
# Let's see what that looks like from the places perspective
q = """
Create a table of places that Alex visits:

In the first column indicate the place name. 
In the 2nd column indicate the frequency he visits them.
In the 3rd column list the people who were there with him by strength of the bond of the people.
In the 4th column indicate the importance of the place.
In the 5th column describe the location in 255 characters or less.
In the 6th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""
await process_and_display_RAG_comparison(q)

------ STD RAG -------------------------

<think>
Okay, I need to create a table based on Alex's visits from the provided document. Let me read through the document carefully to extract all the relevant information about Alex's locations.

First, Alex is in "Dulce" multiple times and mentions it's his base. He also refers to "the celestial divide," which might be an external location beyond Dulce. There's a scene where agents meet at a place described as "sterile hum and flickering lights of Dulce Base's command center." 

Additionally, Alex is in "the sentience" — this seems like a metaphorical or abstract location related to the data they're processing.

I'll list each unique place he visits. For frequency, I'll count how many times each place appears. The third column will note who was present based on bond strength—probably limited to significant team members like Jordan and Taylor. 

Importance could be subjective but I'll assume each location has varying significance: Dulce as a 

Here's the table summarizing Alex's visits based on the provided document:

| Place Name | Frequency (Times Visited) | People Present (Bond Strength) | Importance of Place | Description in 255 Characters | Total Weight of Interactions |
|------------|---------------------------|----------------------------------|-----------------------|-------------------------------|------------------------------|
| Dulce       | Multiple                 | Jordan, Taylor                  | High                  | Base for MIU operations and data processing.  | High (mentioned multiple times) |
| Celestial Divide | Once or reference      | N/A                              | Medium                | Mentioned as a boundary beyond.  | Medium                        |
| Sentience   | Once                     | None                             | Abstract              | Metaphorical location for data.    | Low                          |

This table captures Alex's visits, noting where he spends time, who is present, and the overall significance of each location based on context from the document.

</content>
------ GRAPH RAG -------------------------

<think>
Alright, so I'm trying to figure out how to create this table for Alex's visits based on the given information. First, I need to parse all the data and extract the necessary details about where Alex has been, how often he goes there, who he's been with him, the importance of each place, a brief description of each location, and the total weight of his interactions.

Looking at the knowledge graph provided, I see that Alex is part of the Paranormal Military Squad. He's been mentioned in several scenes, mostly within Dulce Base during Operation: Dulce. There's also mention of a conflict between Taylor Cruz and Jordan Hayes, but since Alex is not involved in those conflicts, I won't include him as someone present there.

So, from the data:
1. Alex visits Dulce Base multiple times—specifically mentioned five times.
2. He doesn't visit any other places like Washington or Sam's headquarters (Dulce Base).
3. Since he hasn't been w

Here is the table summarizing Alex's visits based on the extracted information:

| Place Name          | Frequency of Visit | People Present            | Importance      | Description of Location                          | Total Weight of Interactions |
|---------------------|--------------------|---------------------------|-----------------|---------------------------------------------------|------------------------------|
| Dulce Base         | 5                  | None                      | 10              | High-tech security facility with advanced tech.    | 5                            |

This table focuses solely on Alex's primary location, Dulce Base, as he has not visited any other places mentioned in the provided context. The interactions are counted based on his presence during Operation: Dulce without any additional people present or conflicts involved.

</content>


In [103]:
#Let's try a creative endeavor
q = """
    Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Describe this cover in 20 words or less.
"""
await process_and_display_RAG_comparison(q)


------ STD RAG -------------------------

<think>
Okay, so I'm trying to figure out which scene from the given document would be the most emotionally charged and important for making it onto a book cover. The user mentioned that Alex Mercer has strong relationships and that certain places are more important to the plot. 

First, I'll read through the document carefully to identify key moments involving Alex and his interactions with others or significant events in the story. From the text, I see scenes where Alex interacts with Dr. Jordan Hayes, Sam Rivera, and Taylor Cruz. Each of these characters has their own dynamics.

Looking at the first part of the document, Alex is shown as someone who's invested in understanding alien codes but also emphasizes the importance of interpreting those signals correctly, especially regarding intent. This suggests a moment where there's both intellectual engagement and a sense of responsibility or fear, which could be emotionally charged.

In another

Alex Mercer stands as a pivotal character, balancing intellectual engagement with caution, making his pivotal moment of contact emotionally charged.

</content>
------ GRAPH RAG -------------------------

<think>
Okay, so I need to help create a cover for a book based on Alex Mercer's strongest relationships and the most important places from the plot. The user also wants it to be emotionally charged and important enough to make the cover of the book. Let me break this down step by step.

First, looking at the knowledge graph provided, there are four agents: Alex M., Jordan H., Sam R., and Taylor C. Each has various relationships with each other, but I'm focusing on Alex's strongest relationships. From the data:

- Alex is a member of the Paranormal Military Squad (PMS).
- He leads the team during Operation: Dulce.
- He mentors Sam, which shows his leadership and role within the team.
- He works within Dulce Base.

So the key elements here are:
1. Alex as a leader
2. His role in the operation
3. Dulce Base

Now, considering the cover needs to be emotionally charged. Emotionally charged covers often evoke strong emotions—something th

**Cover Description:**  
A dark, foreboding scene where a lone figure stands firm at the center of an abandoned military base. The atmosphere is tense, with elements hinting at hidden dangers and unknown threats, creating an eerie mood that emphasizes suspense and tension.

</content>


In [60]:


q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the style of an 8-bit video game.
"""

await process_and_display_RAG_comparison(q)





 ------ STD RAG -------------------------





 ------ Think -------------------------

Alright, let's break down how I arrived at the answer for the user's question about creating a cover style based on Alex Mercer's strongest relationships and key places that matter for the plot, using an 8-bit video game-style cover. 

First, I started by reading through the provided documents to understand the context of the story. It seems it's a book set in a dystopian future where humanity has been trying to contact extraterrestrial beings for centuries without success. The main character is Dr. Jordan Hayes, who decipheres messages from other worlds but faces a dangerous situation when she uncovers ancient technologies.

The user wants me to choose a scene based on Alex Mercer's relationships and the most important plot points. Alex is the protagonist, so I should focus on scenes that highlight his development and key interactions. His relationships with Dr. Jordan Hayes (the doctor) and Dr

Based on your query, here’s my response:

**Answer:**

To create a scene that is both emotionally charged and important for the plot, and to draw it in an 8-bit video game-style style, I would recommend using **Alex Mercer’s office at Dulce Base's command center**. This location is where Alex (Dr. Jordan Hayes) and Sam (Dr. Sam Rivera) work together as scientists and human computers. Their physical presence in the room reflects their technical expertise and teamwork, while their interactions with other characters add depth to the story.

Here’s a breakdown of why this scene fits your criteria:

1. **Emotional Depth**:  
   Alex’s emotional journey is central to the story—his determination to communicate with extraterrestrial beings despite being trapped in a dystopian world, his internal conflict between science and humanity, and his personal growth as he learns to trust his instincts. His interactions with Sam and other characters in the room emphasize these themes.

2. **Key Plot Points**:  
   - The passage of time at Dulce Base’s command center highlights the passage of the book and Alex’s journey through the unknown.  
   - The city itself (Dulce Base) is a gritty, oppressive place that represents the dystopian setting of the story and adds to its emotional weight.

3. **8-bit Video Game Style**:  
   - The cover will be minimalistic with thick lines to create a fast-paced sense of time.  
   - The colors will reflect the dark, oppressive feel of the city while still being recognizable and detailed.  
   - The overall aesthetic should resemble that of an 8-bit video game cover, with vibrant but not over-the-top graphics.

Here’s how I would structure this scene for your book:

---

**Cover Style: 8-Bit Video Game**

- **Image Layout**:  
  - A grid-based design representing the fast-paced nature of the story.  
  - Thick lines representing time moving quickly and action taking place.  
  - The city itself will have darker tones, neon colors, and a gritty atmosphere.

- **Alex’s Role**:  
  - Alex will be depicted as someone struggling with his internal conflict—his determination to communicate with extraterrestrial beings while being trapped in the past.  
  - His face will be expressive, showing vulnerability and emotional turmoil.

- **Sam’s Role**:  
  - Sam will be portrayed as a scientist who is meticulous in decoding messages but also has an unfeeling nature that can cause stress for Alex.  
  - Their interaction highlights Sam’s expertise while also showing his potential pitfalls.

- **Text on the Back**:  
  - A bold, text-based quote reflecting Alex’s perspective: *"I was once a computer. Then I became a scientist. But now, I’m both."*

---

This scene captures the essence of your request and aligns with the emotional and technical themes of the story while maintaining an 8-bit video game-style aesthetic.





 ------ GRAPH RAG -------------------------





 ------ Think -------------------------

Alright, I need to figure out how to respond to this user's query about creating a cover for a book featuring Alex Mercer's role in a 2019 movie. The user provided some background information about the movie and the characters' relationships within the movie, including key roles like Alex acting as an outsider due to his computer skills.

The user wants me to create a scene based on this movie that is emotionally charged and important for the cover style of an 8-bit video game. They also want the scene to represent this role well.

First, I'll analyze the information given about the characters. Alex Mercer is seen as someone without much traditional fame but with unique skills that make him stand out. He leads a team that collaborates on a project which he refers to as "Operation Dulce." The movie has a high budget and relies heavily on Alex's expertise in computer games.

The key relationship

To create a scene that captures Alex Mercer's role within the movie "The Shape of Water" (2019) and sets it apart from its competitors, we need to focus on a scene that emphasizes his leadership style as an outsider figure who takes on a high-pressure role. The key relationships within the movie are:

- **Alex Mercer**: The star-studded cast is at odds with each other, but Alex consistently emerges as the clear leader and protector of others. He mentors Jordan, a scientist, and Sam, another technician, working together around him.

Given these dynamics for an 8-bit video game-style cover, here's a scene idea:

---

### **Scene Title**: *The Shape Shifts in 2019*  

This is the opening cut to the book's cover. The scene shows Alex Mercer as a fresh-faced hero leading a group of four: Jordan (a scientist working on something obscure), Sam (a tech-savvy technician under his wing), and their team mate J.D., who has been working hard for weeks but is finally waking up to the reality of their situation.

#### **Visual Style**:
- The background is neon-lit with bright colors that evoke the rugged, action-packed vibe of 8-bit gaming.  
- The scene uses bold, vibrant colors and a dynamic layout to emphasize tension and drama.
- Alex's expression shows determination and grace as he leads the team through a challenging but ultimately triumphing mission.

---

### **Step-by-Step Scene Description**:

1. **Opening Sequence**:
   - Start with Alex Mercer in a clean, white room, looking resolute. His facial hair is neatly trimmed, and his eyes are focused on the screen.
   - The background is a dimly lit room filled with neon lights from the game's theme music, creating a gritty, high-tech feel.

2. **Introducing the Team**:
   - Jordan appears as a scientist in a slightly moreubtled outfit. He stands at the center of the room and looks out at Alex.
   - Sam is under Jordan's wing but has his own story to tell. He works alongside Jordan, maintaining their bond while looking on the horizon.
   - J.D. is standing behind them, managing some of the team's backcountry activities.

3. **The Challenge**:
   - The group faces a high-stakes mission in front of the camera. It’s a low-level platformer where they must navigate rough terrain to reach a hidden goal while avoiding traps and enemies.
   - Each member of the team struggles but stays focused on their role, working together to overcome the odds.

4. **Alex’s leadership**:
   - Alex emerges as the leader in this moment, quickly joining Jordan, Sam, and J.D. behind him.
   - He looks out at them with a confident expression, knowing that they have reached a point of no return.
   - Jordan steps forward to lead the group, demonstrating his mentorship and dedication.

5. **Conflict and Triumph**:
   - The team faces obstacles that test their abilities but remain manageable together.
   - J.D. and Sam work behind the camera to keep the action moving while Alex leads the pack, guiding the group through tough terrain.
   - The mission is a success, and all four characters return to their normal lives.

6. **The End**:
   - After completing the final jump over the pit, the group returns to the main room with Alex looking on.
   - Alex waves goodbye to Jordan, Sam, and J.D., highlighting their strong bond as a team despite their differences.

---

### **Key Elements for the Scene**:
- **Leadership**: Alex leads without being arrogant or dominant. He uses his unique skills (his computer game knowledge) to lead a group that’s usually seen as “out of their league.”
- **Mentorship**: The scene shows Jordan and Sam learning from Alex, but not taking it lightly.
- **Collaboration**: The team works together seamlessly, trusting each other despite their differences.
- **Drama**: The mission is a high-stakes, action-packed showdown that emphasizes the characters’ unity.

---

### **Cover Title**: *The Shape of Water*  

This scene ties into Alex Mercer’s role as a fresh-faced hero in a highly competitive world. It highlights his leadership style and the importance of collaboration within a tight-knit group. The neon-lit colors and dynamic layout set it apart from other 8-bit gaming covers, making it feel like it’s right for this movie.

--- 

This scene should capture Alex Mercer's role as an outsider figure in a high-stakes group and emphasize his leadership and mentorship style within the movie.